In [77]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [78]:
cd /content/gdrive/My Drive/Colab Notebooks/data

/content/gdrive/My Drive/Colab Notebooks/data


In [79]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [80]:
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader


# Read the train/test file from kaggle
train = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/data/train_covid.csv')
test = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/data/test_covid.csv')

In [81]:
# Fill missing value by average
train["new_cases"]=train["new_cases"].fillna(train["new_cases"].mean())
train["new_deaths"]=train["new_deaths"].fillna(train["new_deaths"].mean())
train["stringency_index"]=train["stringency_index"].fillna(train["stringency_index"].mean())
train["population"]=train["population"].fillna(train["population"].mean())
train["population_density"]=train["population_density"].fillna(train["population_density"].mean())
train["median_age"]=train["median_age"].fillna(train["median_age"].mean())
train["aged_65_older"]=train["aged_65_older"].fillna(train["aged_65_older"].mean())
train["aged_70_older"]=train["aged_70_older"].fillna(train["aged_70_older"].mean())
train["female_smokers"]=train["female_smokers"].fillna(train["female_smokers"].mean())
train["male_smokers"]=train["male_smokers"].fillna(train["male_smokers"].mean())
train["gdp_per_capita"]=train["gdp_per_capita"].fillna(train["gdp_per_capita"].mean())
train["handwashing_facilities"]=train["handwashing_facilities"].fillna(train["handwashing_facilities"].mean())
train["hospital_beds_per_thousand"]=train["hospital_beds_per_thousand"].fillna(train["hospital_beds_per_thousand"].mean())
train = train.fillna(train.mean())
test = test.fillna(train.mean())

In [82]:
## 2layer FC layer NN
class myModel(torch.nn.Module):
    def __init__(self):
        super(myModel, self).__init__()
        self.model = torch.nn.Sequential(
                        torch.nn.Linear(30, 10),
                        torch.nn.GELU(),
                        torch.nn.Linear(10, 2),
                        torch.nn.ReLU()
        )
        # [c*7, d*7] --> [c, d]
        
    def forward(self, x):
        c_and_d = self.model(x)
        c = c_and_d[:, 0] # new_case
        d = c_and_d[:, 1] # new_death
        
        #print("c, d: ", c, d)
        return c, d

class myDataset(Dataset):
    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs
        
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {"inputs": self.inputs[idx], "outputs": self.outputs[idx]}
    
def my_collate(samples):
    inputs = []
    outputs = []
    for sample in samples:
        inputs.append(sample["inputs"])
        outputs.append(sample["outputs"])
        
    return {"inputs": torch.tensor(inputs), "outputs": torch.tensor(outputs)}

In [83]:
# initialize the dataset 
train_case = [0.] * 10
train_death = [0.] * 10
train_age = [0.] * 10
train_input = train_case + train_death + train_age
train_inputs = []
train_outputs = []
locations = []


test_case = [0.] * 10
test_death = [0.] * 10
test_age = [0.] * 10
test_input = test_case + test_death + test_age
test_inputs = []

In [84]:
# Make dataset for training 
# [c*7, d*7] --> [c, d]
# (ex: given: 4/1~4/7 case & death, predict: 4/8 case & death)
for key in tqdm(sorted(set(train["location"]))):
    locations.append(key)
    temp_df = train[train["location"] == key]
    train_case = [0.] * 10
    train_death = [0.] * 10 
    train_age = [0.] * 10

    train_input = train_case + train_death + train_age
    train_inputs.append(train_input)
    
    append_flag = 1
    for index, row in temp_df.iterrows():
        if(append_flag):
            train_outputs.append([float(row["new_cases"]), float(row["new_deaths"])])
            append_flag = 0
            
        train_case = train_case[1:] + [float(row["new_cases"])]
        train_death = train_death[1:] + [float(row["new_deaths"])]
        train_age = train_age[1:] + [float(row["median_age"])]
        train_input = train_case + train_death + train_age
        #train_inputs.append(train_input)
        
        # To handle too many multiple values & the last value
        # (ex: [0, 0, .... , 0, 0] --> [0, 0])
        if train_input in train_inputs or index == temp_df["id"].values[-1]: continue
        else: train_inputs.append(train_input); append_flag = 1

100%|██████████| 208/208 [00:06<00:00, 30.29it/s]


In [85]:
# Evaluation dataset
test_input_dict = {}
for key in tqdm(sorted(set(train["location"]))):
    
    # To predict 5/1 
    temp_df = train[train["location"] == key]
    temp_df = temp_df[(temp_df['date'] >= '2020-04-21') & (temp_df['date'] <= '2020-04-30')]
    
    
    test_case = list(map(lambda x: float(x), temp_df["new_cases"].values))
    test_death = list(map(lambda x: float(x), temp_df["new_deaths"].values))
    test_age = list(map(lambda x: float(x), temp_df["median_age"].values))
    
    if(len(test_case) < 10): test_case = [0.] * (10 - len(test_case)) + test_case
    if(len(test_death) < 10): test_death = [0.] * (10 - len(test_death)) + test_death
    if(len(test_age) < 10): test_age = [0.] * (10 - len(test_age)) + test_age    
   
    test_input = test_case + test_death + test_age
    
    test_input_dict[key] = test_input

100%|██████████| 208/208 [00:00<00:00, 396.34it/s]


In [86]:
batch_size = 100

## Dataset Load
dataset = myDataset(train_inputs, train_outputs)
params = {'batch_size': batch_size,
          'shuffle': True,
          'num_workers':0,
          'drop_last': True,
          'collate_fn': my_collate
         }
train_generator = DataLoader(dataset, **params)

## Model load
model = myModel()

# Loss/Optimizer define
optimizer = torch.optim.AdamW(model.parameters(), amsgrad = True)

class RMSLELoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = torch.nn.MSELoss()
        
    def forward(self, pred, actual):
        return torch.sqrt(self.mse(torch.log(pred + 1), torch.log(actual + 1)))
    
# Loss
criterion = RMSLELoss()

In [87]:
# Training
model.train()
for epoch in tqdm(range(100)):
    for k in train_generator:
        pred_c, pred_d = model(k["inputs"])
        
        loss_c = criterion(pred_c, k["outputs"][:, 0])
        loss_d = criterion(pred_d, k["outputs"][:, 1])
        total_loss = (loss_c)*0.9  + (loss_d) * 0.1
        
        
        total_loss.backward()
        torch.nn.utils.clip_grad_value_(model.parameters(), 1)
        optimizer.step()
        
    print("epoch: ", total_loss)

  1%|          | 1/100 [00:00<00:20,  4.85it/s]

epoch:  tensor(1.1868, grad_fn=<AddBackward0>)


  2%|▏         | 2/100 [00:00<00:20,  4.77it/s]

epoch:  tensor(0.8536, grad_fn=<AddBackward0>)


  3%|▎         | 3/100 [00:00<00:20,  4.71it/s]

...

epoch:  tensor(0.7371, grad_fn=<AddBackward0>)


 99%|█████████▉| 99/100 [00:23<00:00,  4.29it/s]

epoch:  tensor(0.7631, grad_fn=<AddBackward0>)


100%|██████████| 100/100 [00:23<00:00,  4.30it/s]

epoch:  tensor(0.9016, grad_fn=<AddBackward0>)


In [88]:
## Evaluation
eval_location = []

# save all predictions
predict_case = []
predict_death = []

model.eval()
for idx, row in test.iterrows():
    if(row["location"] not in eval_location):
        #print(row["location"])
        eval_location.append(row["location"])
        eval_input = test_input_dict[row["location"]]
        
    c, d = model(torch.tensor(eval_input).unsqueeze(0))
    predict_case.append(c)
    predict_death.append(d)
    
    # update the last new_case/new_death value
    eval_input[9] = c
    eval_input[19] = d

In [89]:
# Make submission file
predict_all = torch.stack([torch.tensor(predict_case), torch.tensor(predict_death)], dim = 1)
ppredict = predict_all.numpy()
ppredict = pd.DataFrame(ppredict, columns = ["new_cases", "new_deaths"])
ppredict["id"] = ppredict.index

ppredict = ppredict[["id", "new_cases", "new_deaths"]]

In [90]:
# Save submission file
ppredict.to_csv("Submit.csv", index = False)